In [ ]:
import os
import win32com.client as win32
from win32com.client import constants

import pandas as pd
import docx
from docx.api import Document
from tqdm.notebook import tqdm

In [ ]:
def to_docx_format(document_path):
    if os.path.splitext(os.path.abspath(document_path))[1] in ['.doc', '.DOC']:
        document_path = os.path.abspath(document_path)
        
        word = win32.gencache.EnsureDispatch('Word.Application')
        document = word.Documents.Open(document_path)
        document.Activate()
    
        new_document_path = os.path.splitext(os.path.abspath(document_path))[0] + '.docx'
        word.ActiveDocument.SaveAs(new_document_path, FileFormat=constants.wdFormatXMLDocument)
        document.Close(False)
    
    return os.path.splitext(os.path.abspath(document_path))[0] + '.docx'

In [ ]:
def document_paths_for_year(year):
    data_path = f'./{year}'
    document_paths = []
    
    for folder in os.listdir(data_path):
        federal_region_path = os.path.join(data_path, folder)
        if os.path.isdir(federal_region_path):
            for file_name in os.listdir(federal_region_path):
                document_path = to_docx_format(os.path.join(federal_region_path, file_name))
                document_paths.append(document_path)
    
    return document_paths

In [ ]:
def find_values(document_path, indicator):
    indicator = indicator.lower()
    document = Document(document_path)
    for table in document.tables:
        for row_number, row in enumerate(table.rows):
            for cell in [docx.table._Cell(tc, table) for tc in row._tr.tc_lst]:
                if cell.text.lower().find(indicator) >= 0:
                    cells = [docx.table._Cell(tc, table) for tc in table.rows[row_number + 2]._tr.tc_lst]
                    for index, entry in enumerate(cells):
                        if entry.text.lower()[0].isdigit() or entry.text.lower()[0] == '-':
                            year = int(os.path.split(os.path.split(os.path.split(document_path)[0])[0])[1])
                            if year == 2020:
                                return dict([(year - 3 + i, cells[j].text[:5]) for i, j in enumerate(range(index, index + 3))])
                            elif year >= 2012 and year <= 2018:
                                return dict([(year - 2 + i, cells[j].text[:5]) for i, j in enumerate(range(index, index + 2))])
                            else:
                                return dict([(year - 2 + i, cells[j].text[:5]) for i, j in enumerate(range(index, index + 1))])
    return {}

In [ ]:
def find_city(city, text):
    for parts in text.split(' '):
        for part in parts.split('\n'):
            if part.lower().find(city.lower()) == 0:
                return True
    return False

In [ ]:
def get_indicator_values(cities, document_paths, indicator):
    values_dict = dict([(city, {}) for city in cities])
    
    for document_path in tqdm(document_paths):
        document = Document(document_path)
        text = ''.join([cell.text.lower() for table in document.tables for row in table.rows for cell in [docx.table._Cell(tc, table) for tc in row._tr.tc_lst]])
        
        not_found = True
        for city in cities:
            if find_city(city[:int(0.8*len(city)) + 1], document_path.split('_')[-1]):
                values_dict[city] = find_values(document_path=document_path, indicator=indicator)
                not_found = False
                break
        if not_found:
            for city in cities:
                if len(values_dict[city]) == 0 and find_city(city[:int(0.8*len(city)) + 1], text):
                    values_dict[city] = find_values(document_path=document_path, indicator=indicator)
                    break
            
    return values_dict

In [ ]:
cities = list(pd.read_excel('Исх данные Города РАНХиГС_2021.xlsx', sheet_name='13.Мощность АПУ')['Наименование города'].dropna())
if 'В среднем по Российской Федерации' in cities:
    cities.remove('В среднем по Российской Федерации')

In [ ]:
def merge_dicts(dicts):
    for key in tqdm(dicts[0].keys()):
        dicts[0][key].update(dicts[1][key])

In [ ]:
values = dict([(city, {}) for city in cities])

In [ ]:
for name in os.listdir('./data'):
    year = int(name)
    print(year)
    values_dict = get_indicator_values(cities=cities,
                                       document_paths=document_paths_for_year(year=os.path.join('./data', name)),
                                       indicator='коек')
    merge_dicts([values, values_dict])

In [ ]:
pd.DataFrame(data=[[values[city].get(year) for year in range(2002, 2020)] for city in cities],
             index=cities,
             columns=[year for year in range(2002, 2020)]).to_excel('коек_1.xlsx')